In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
pip install sdv --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.3/154.3 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 96.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.4/52.4 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.1/179.1 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 65.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.2/83.2 kB 4.4 MB/s eta 0:00:00


In [ ]:
import sdv
import os
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [ ]:
print(sdv.__version__)

1.17.2


In [ ]:
from sdv.single_table import GaussianCopulaSynthesizer
from sdv.single_table import CTGANSynthesizer
from sdv.single_table import TVAESynthesizer
from sdv.single_table import CopulaGANSynthesizer

from sdv.metadata import SingleTableMetadata
from sdv.sampling import Condition as cd

In [ ]:
import pickle
# Load object from file
with open(gdrivePath +os.sep + "data" +os.sep+'X_train.pkl', 'rb') as f:
    X_train = pickle.load(f)
with open(gdrivePath +os.sep + "data" +os.sep+'X_test.pkl', 'rb') as f:
    X_test = pickle.load(f)
with open(gdrivePath +os.sep + "data" +os.sep+'y_train.pkl', 'rb') as f:
    y_train = pickle.load(f)
with open(gdrivePath +os.sep + "data" +os.sep+'y_test.pkl', 'rb') as f:
    y_test = pickle.load(f)


In [ ]:
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(11337, 7514) (11337,) (2835, 7514) (2835,)


In [ ]:
unique_values, counts = np.unique(y_train, return_counts=True)
display(dict(zip(unique_values, counts)),np.max(counts))

{'PSC': 42,
 'acinar': 1090,
 'activated_stellate': 227,
 'alpha': 3897,
 'beta': 2950,
 'delta': 759,
 'ductal': 1360,
 'endothelial': 231,
 'epsilon': 17,
 'gamma': 339,
 'macrophage': 44,
 'mast': 20,
 'mesenchymal': 64,
 'pp': 148,
 'quiescent_stellate': 138,
 'schwann': 11}

3897

In [ ]:
X_train_pd = pd.DataFrame(X_train)    # values
             #index=data[1:,0],    # 1st column as index
             #columns=data[0,1:])

In [ ]:
np.median(counts)

187.5

In [ ]:
np.mean(counts)

708.5625

In [ ]:
Q1, Q2, Q3 = np.quantile(counts, [0.25, 0.5, 0.75], axis=0, method='nearest')
print("Q1",Q1,"\nQ2",Q2,"\nQ3", Q3)


Q1 44 
Q2 227 
Q3 759


In [ ]:
from sdv.metadata import SingleTableMetadata
metadata = SingleTableMetadata()
metadata.detect_from_dataframe(X_train_pd)
#metadata.set_primary_key(column_name='gene')

TVAE

In [ ]:
unique_values, counts = np.unique(y_train, return_counts=True)
display(dict(zip(unique_values, counts)))

{'PSC': 42,
 'acinar': 1090,
 'activated_stellate': 227,
 'alpha': 3897,
 'beta': 2950,
 'delta': 759,
 'ductal': 1360,
 'endothelial': 231,
 'epsilon': 17,
 'gamma': 339,
 'macrophage': 44,
 'mast': 20,
 'mesenchymal': 64,
 'pp': 148,
 'quiescent_stellate': 138,
 'schwann': 11}

In [ ]:
from sdv.single_table import TVAESynthesizer

def tVAE_method(X_min,num_synthetic_samples, metadata):
  tvae = TVAESynthesizer(metadata,epochs=50)
  tvae.fit(X_min)
  # Create synthetic data
  synthetic_data = tvae.sample(num_synthetic_samples)
  return synthetic_data

In [ ]:
# Get unique values and their counts
unique_values, counts = np.unique(y_train, return_counts=True)
classlabel_counts = dict(zip(unique_values, counts))
max_count = 759 #np.max(counts)

i=1
for label, count in classlabel_counts.items():
    #print(x.shape, y.shape)
    #print("label, count, max_count",label,count,max_count)
    X_minority = X_train_pd[y_train == label]
    if count < max_count:
        #print("\n")
        print(f"Value {label} appears {count} times.")
        num_synthetic_samples = max_count - count
        synthetic_samples = tVAE_method(X_minority, num_synthetic_samples,metadata )

        X_minority = np.array(X_minority)
        synthetic_samples = np.array(synthetic_samples)
        if i==1:
          X_train_gen = X_minority
          X_train_gen = np.vstack([X_train_gen, synthetic_samples])

          y_train_gen = np.full(759, label)

          y_train_indexes = np.full(count, 1)
          y_train_indexes = np.concatenate([y_train_indexes, np.full(num_synthetic_samples, 2)])
        else:
          tmp = np.vstack([X_minority, synthetic_samples])
          X_train_gen = np.vstack([X_train_gen, tmp])
          y_train_gen = np.concatenate([y_train_gen, np.full(759, label)])

          y_train_indexes = np.concatenate([y_train_indexes, np.full(count, 1)])
          y_train_indexes = np.concatenate([y_train_indexes, np.full(num_synthetic_samples, 2)])
    else:
        X_train_gen = np.vstack([X_train_gen, X_minority])
        y_train_gen = np.concatenate([y_train_gen, np.full(count, label)])
        y_train_indexes = np.concatenate([y_train_indexes, np.full(count, 1)])
    i=i+1

Value PSC appears 42 times.


/usr/local/lib/python3.10/dist-packages/sdv/single_table/base.py:119: FutureWarning: The 'SingleTableMetadata' is deprecated. Please use the new 'Metadata' class for synthesizers.
  warnings.warn(DEPRECATION_MSG, FutureWarning)
/usr/local/lib/python3.10/dist-packages/sdv/single_table/base.py:104: UserWarning: We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.
  warnings.warn(


Value activated_stellate appears 227 times.


/usr/local/lib/python3.10/dist-packages/sdv/single_table/base.py:119: FutureWarning: The 'SingleTableMetadata' is deprecated. Please use the new 'Metadata' class for synthesizers.
  warnings.warn(DEPRECATION_MSG, FutureWarning)


Value endothelial appears 231 times.


...

/usr/local/lib/python3.10/dist-packages/sdv/single_table/base.py:119: FutureWarning: The 'SingleTableMetadata' is deprecated. Please use the new 'Metadata' class for synthesizers.
  warnings.warn(DEPRECATION_MSG, FutureWarning)


Value quiescent_stellate appears 138 times.


/usr/local/lib/python3.10/dist-packages/sdv/single_table/base.py:119: FutureWarning: The 'SingleTableMetadata' is deprecated. Please use the new 'Metadata' class for synthesizers.
  warnings.warn(DEPRECATION_MSG, FutureWarning)


Value schwann appears 11 times.


/usr/local/lib/python3.10/dist-packages/sdv/single_table/base.py:119: FutureWarning: The 'SingleTableMetadata' is deprecated. Please use the new 'Metadata' class for synthesizers.
  warnings.warn(DEPRECATION_MSG, FutureWarning)


In [ ]:
X_train_gen.shape, y_train_gen.shape, X_test.shape, y_test.shape

((18405, 2500), (18405,), (2835, 2500), (2835,))

In [ ]:
file_name ='TVAE'

In [ ]:
import pickle

# Save object to file
with open(gdrivePath +os.sep+"results"+os.sep+"TVAE"+os.sep+file_name+'_X_train.pkl', 'wb') as f:
    pickle.dump(X_train_gen, f)

with open(gdrivePath +os.sep+"results"+os.sep+"TVAE"+os.sep+file_name+'_X_test.pkl', 'wb') as f:
    pickle.dump(X_test, f)

with open(gdrivePath +os.sep+"results"+os.sep+"TVAE"+os.sep+file_name+'_y_train.pkl', 'wb') as f:
    pickle.dump(y_train_gen, f)

with open(gdrivePath +os.sep+"results"+os.sep+"TVAE"+os.sep+file_name+'_y_test.pkl', 'wb') as f:
    pickle.dump(y_test, f)

with open(gdrivePath +os.sep+"results"+os.sep+"TVAE"+os.sep+file_name+'_y_train_idx.pkl', 'wb') as f:
    pickle.dump(y_train_indexes, f)